<a href="https://colab.research.google.com/github/ufrpe-mineracao-textos/projeto-de-mineracao-20192-ner-analise-juridica/blob/master/sentence_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificação de sentenças utilizando o algoritmo AdaBoost

In [0]:
import spacy
import json
import os
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
os.system("python -m spacy download pt")

0

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Caminho utilizado por Fernanda:

In [0]:
%cd drive/My\ Drive/Projeto\ de\ MT

/content/drive/My Drive/Projeto de MT


Caminho utilizado por Saulo:

In [0]:
%cd drive/My\ Drive/Academico/8º\ Período/Mineração\ de\ Texto/Projeto\ de\ MT

In [0]:
nlp = spacy.load('pt')

with open('Model/mineracao_mulheres.json') as file_data:
    data = json.load(file_data)

## Preparando a base
Neste passo, é necessário obter os índices da localização dos decretos rotulados via Dataturks.

In [0]:
points = {}
contents = []
decretos = []

for index, item in enumerate(data):
    has = False
    point = []
    contents.append(item['content'])
    
    for note in item['annotation']:
        if note['label'][0] == "decreto":
            point = [note['points'][0]['start'], note['points'][0]['end']]
            decretos.append(note['points'][0]['text'])
            if str(index) in contents:
                aux = points[str(index)]
                aux.append(point)
                points[str(index)] = aux
            else:
                points[str(index)] = [point]

print(f"{len(decretos)} decretos recuperados.")

160 decretos recuperados.


## Rotular as sentenças que contém um decreto

In [0]:
sentences_train = []
labels_train = []

for index, text in enumerate(contents):
    doc = nlp(text)
    if str(index) in points:
        found = False
        
        for sentence in doc.sents:
            if found: 
                break
            
            for token in sentence:
                if (token.idx >= points[str(index)][0][0]
                      and token.idx <= points[str(index)][0][1]):
                    aux = str(sentence)
                    aux = aux.replace('  ', '')
                    aux = aux.replace('\n', '')
                    aux = aux.replace('\r', '')
                    
                    if aux.isalpha:
                        sentences_train.append(aux)
                        labels_train.append(1)
                    
                    found = True
                    break
    else:
        for sentence in doc.sents:
            aux = str(sentence)
            aux = aux.replace('  ', '')
            aux = aux.replace('\n', '')
            aux = aux.replace('\r', '')
            
            if aux.isalpha:
                sentences_train.append(aux)
                labels_train.append(0)

## Treinando a base

In [0]:
tfidf = TfidfVectorizer(smooth_idf=True, norm='l1', max_features=1000)
x_train_tfidf = tfidf.fit_transform(sentences_train)

clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_train_tfidf, labels_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=0)

## Testando a base

In [0]:
with open('Database/Dados para teste/andamentos.json') as file_data:
    data_test = json.load(file_data)

In [0]:
count = 0
sentences_test = []

for idx, item in enumerate(data_test):
    doc = nlp(item['texto'])
    doc_test = [str(sent) for sent in doc.sents if len(str(sent)) > 20]
    doc_tfidf = tfidf.transform(doc_test)
    predict = clf.predict(doc_tfidf)
    for idx2, label in enumerate(predict):
        if label:
            print("#"*30)
            print(f"Decreto do andamento {idx}")
            print(f"{doc_test[idx2]}\n")
            count += 1
            break

    if count == 50: break

##############################
Decreto do andamento 4
Aguarde-se o envio da peça informativa e a manifestação do Ministério Público, através de sua Central de Inquéritos, nos prazos previstos nos arts.

##############################
Decreto do andamento 5
A meu ver, há de se frisar que a prisão preventiva não se afigura como prisão pena, mas sim como prisão processual.

##############################
Decreto do andamento 6
A meu ver, há de se frisar que a prisão preventiva não se afigura como prisão pena, mas sim como prisão processual.

##############################
Decreto do andamento 7
Desta forma, DESIGNO audiência de instrução e julgamento para o dia 06 de maio de 2019, às 14h , devendo a Secretaria providenciar as intimações e requisições necessárias.
	

##############################
Decreto do andamento 9
Aguarde-se o envio da peça informativa e a manifestação do Ministério Público, através de sua Central de Inquéritos, nos prazos previstos nos arts.

#######################